In [1]:
import os
import csv
import pandas as pd
import numpy as np
from PIL import Image
import _pickle as pickle

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.autograd import Variable
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
is_cuda = torch.cuda.is_available()
num_gpu = 1
iters = 10
batch_size = 32
load_model = False
train_path = '../../Datasets/ChineseDetection/train'
test_path = '../../Datasets/ChineseDetection/test1'
# test_labels = csv.reader(open('../../Datasets/ChineseDetection/label-test1-fake.csv'))
test_labels = pd.read_csv('../../Datasets/ChineseDetection/label-test1-fake.csv', header=0)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # transforms.Normalize((0.5), (1.0))
])
LABELS = ['且', '世', '东', '九', '亭', '今', '从', '令', '作', '使', '侯', '元', 
          '光', '利', '印', '去', '受', '右', '司', '合', '名', '周', '命', '和', 
          '唯', '堂', '士', '多', '夜', '奉', '女', '好', '始', '字', '孝', '守',
          '宗', '官', '定', '宜', '室', '家', '寒', '左', '常', '建', '徐', '御', 
          '必', '思', '意', '我', '敬', '新', '易', '春', '更', '朝', '李', '来', 
          '林', '正', '武', '氏', '永', '流', '海', '深', '清', '游', '父', '物',
          '玉', '用', '申', '白', '皇', '益', '福', '秋', '立', '老', '臣', '良',
          '莫', '虎', '衣', '西', '起', '足', '身', '通', '遂', '重', '陵', '雨',
          '章', '高', '黄', '鼎']
LABELDIC, ANTILABELDICT = {}, {}
for i, label in enumerate(LABELS):
    LABELDIC[label] = i
    ANTILABELDICT[i] = label

In [34]:
def default_loader(path):
    return Image.open(path)

class testImageFolder(data.Dataset):
    def __init__(self, rootdir, transform=None, target_transform=None, loader=default_loader):
        images, labels = [], []
        for i in range(test_labels.shape[0]):
            image_name, label = test_labels.loc[i]
            image_path = rootdir + '/' + image_name
            images.append(image_path)

        self.rootdir = rootdir
        self.images = images
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
    
    def __getitem__(self, index):
        sample = self.loader(str(self.images[index]))
        if self.transform is not None:
            sample = self.transform(sample)
        sample = sample.expand(3, 256, 256)
        return sample
    
    def __len__(self):
        return len(self.images)

In [35]:
test_set = testImageFolder(test_path, transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

In [6]:
model = torch.load('test.pkl', map_location='cpu')

/anaconda3/envs/py3/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/anaconda3/envs/py3/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/anaconda3/envs/py3/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the objec

In [58]:
test_set.images[1]

'../../Datasets/ChineseDetection/test1/c875fdacf38eb236d2b759d3397bea49ba3b60ce.jpg'

In [65]:
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    answer_list = []
    for i in range(len(test_set.images)):
        image = Variable(test_set[i])
        image = image.expand([1, 3, 256, 256])
        output = model(image)
        _, pred = output.data.topk(5, dim=1)
        answer_list.append([test_set.images[i], pred.numpy().tolist()[0]])

name = ['filename', 'label']
df = pd.DataFrame(columns=name, data=answer_list, index=0)
df.to_csv('testcsv.csv',encoding='gbk')

KeyboardInterrupt: 

In [69]:
answer_list

[['../../Datasets/ChineseDetection/test1/ae64fde23a2ed7dc3a1b2ecceba2740ea5189ebd.jpg',
  [38, 6, 41, 16, 10]],
 ['../../Datasets/ChineseDetection/test1/c875fdacf38eb236d2b759d3397bea49ba3b60ce.jpg',
  [74, 75, 21, 82, 2]],
 ['../../Datasets/ChineseDetection/test1/ab0fadab1367d2db93e883ccb197a948b65d1563.jpg',
  [89, 80, 38, 61, 82]],
 ['../../Datasets/ChineseDetection/test1/a3ab61411801fd1837076c450349c64492e38afd.jpg',
  [6, 10, 38, 28, 27]],
 ['../../Datasets/ChineseDetection/test1/951f10ac2cab4f33bca8e4389131d194ce51eb49.jpg',
  [25, 44, 12, 77, 26]],
 ['../../Datasets/ChineseDetection/test1/b7d2ff1ae5d7ef03be06b6519737fad06dd83854.jpg',
  [94, 68, 29, 9, 10]],
 ['../../Datasets/ChineseDetection/test1/bc4fa974ef6682b8bb3ff851a57863d6c3744736.jpg',
  [75, 83, 82, 87, 37]],
 ['../../Datasets/ChineseDetection/test1/c11a4fee0d3316c0d384af11d6b0e635daaff3dc.jpg',
  [67, 46, 41, 24, 68]],
 ['../../Datasets/ChineseDetection/test1/7cc4d3d9b10ab3456f9194d852ab55e6638f5be7.jpg',
  [98, 55, 6

In [68]:
name = ['filename', 'label']
df = pd.DataFrame(columns=name, data=answer_list)
df.to_csv('testcsv.csv',encoding='utf8')



In [16]:
test_set.

[59,
 72,
 84,
 17,
 85,
 28,
 46,
 2,
 75,
 3,
 31,
 45,
 79,
 44,
 36,
 77,
 26,
 99,
 73,
 10,
 50,
 47,
 43,
 46,
 5,
 88,
 33,
 64,
 85,
 3,
 91,
 88,
 69,
 19,
 77,
 2,
 82,
 10,
 29,
 72,
 39,
 44,
 2,
 70,
 14,
 59,
 44,
 27,
 50,
 53,
 74,
 13,
 83,
 6,
 37,
 4,
 65,
 11,
 29,
 0,
 7,
 71,
 78,
 20,
 33,
 91,
 85,
 55,
 64,
 43,
 69,
 20,
 90,
 90,
 88,
 70,
 47,
 87,
 29,
 47,
 69,
 10,
 41,
 17,
 69,
 64,
 34,
 10,
 43,
 46,
 92,
 40,
 90,
 35,
 87,
 2,
 15,
 64,
 45,
 75,
 10,
 4,
 68,
 35,
 65,
 31,
 40,
 96,
 78,
 58,
 49,
 37,
 61,
 13,
 53,
 22,
 56,
 42,
 20,
 47,
 98,
 59,
 71,
 31,
 54,
 16,
 52,
 54,
 14,
 46,
 2,
 32,
 62,
 91,
 44,
 20,
 72,
 20,
 87,
 15,
 18,
 47,
 65,
 69,
 20,
 99,
 91,
 46,
 57,
 7,
 96,
 62,
 16,
 95,
 94,
 77,
 43,
 38,
 52,
 1,
 98,
 91,
 20,
 79,
 56,
 68,
 71,
 91,
 85,
 40,
 4,
 90,
 62,
 48,
 62,
 9,
 27,
 11,
 18,
 34,
 36,
 4,
 56,
 18,
 80,
 26,
 62,
 46,
 38,
 26,
 74,
 5,
 24,
 44,
 26,
 39,
 11,
 46,
 9,
 96,
 41,
 29,
 54,
 79,


In [39]:
pred = output.data.max(1)[1]

In [40]:
pred

tensor([74, 32, 56, 53, 80, 32, 79, 24,  8, 24, 58, 51, 26, 85, 94, 18, 32, 31,
        51, 12, 45, 33, 38, 57, 80, 33, 76, 41, 50, 18,  4, 81])

In [43]:
indx_target

tensor([91, 55, 20, 72, 17, 59, 31, 10, 92, 14, 31, 42, 46, 92, 78, 80, 89, 81,
         4, 49, 52, 86, 50, 87, 97, 92, 33, 23, 92, 23, 73,  6])

tensor(0)